<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/100_coin_tarama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import time
import numpy as np

# API Ayarları
COINGECKO_URL = "https://api.coingecko.com/api/v3/coins/markets"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# 100 Coin Verisini Çek
def get_top_100_coins():
    try:
        params = {
            'vs_currency': 'usd',
            'order': 'market_cap_desc',
            'per_page': 100,  # 100 coin
            'page': 1,
            'sparkline': False
        }
        response = requests.get(COINGECKO_URL, headers=HEADERS, params=params, timeout=15)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"API Hatası: {str(e)}")
        return None

# Veriyi İşle
def process_data(data):
    if not data:
        return pd.DataFrame()

    df = pd.DataFrame(data)
    df = df[[
        'id', 'symbol', 'current_price',
        'high_24h', 'low_24h', 'price_change_percentage_24h',
        'market_cap', 'total_volume'
    ]]

    # Türkçe Kolon İsimleri
    df.rename(columns={
        'id': 'Coin ID',
        'symbol': 'Sembol',
        'current_price': 'Fiyat (USD)',
        'high_24h': '24s En Yüksek',
        'low_24h': '24s En Düşük',
        'price_change_percentage_24h': '24s Değişim (%)',
        'market_cap': 'Piyasa Değeri',
        'total_volume': 'Hacim'
    }, inplace=True)

    return df

# Sinyal Üret
def generate_signals(df):
    if df.empty:
        return df

    # Strateji: 24s >%5 ve Hacim > $1M ise AL
    df['Sinyal'] = np.where(
        (df['24s Değişim (%)'] > 5) & (df['Hacim'] > 1e6),
        'AL',
        'BEKLE'
    )

    return df

# Ana Fonksiyon
def scan_coins():
    data = get_top_100_coins()
    if data is not None:
        df = process_data(data)
        if not df.empty:
            df = generate_signals(df)
            return df[['Coin ID', 'Sembol', 'Fiyat (USD)', 'Sinyal']]
    return pd.DataFrame()

# Çalıştır
if __name__ == "__main__":
    while True:
        print(f"\n--- Son Tarama: {time.ctime()} ---")
        results = scan_coins()
        if not results.empty:
            print(results.to_string(index=False))
        else:
            print("Veri alınamadı!")
        time.sleep(300)  # 5 dakikada bir tara


--- Son Tarama: Mon Feb  3 18:28:01 2025 ---
                  Coin ID      Sembol  Fiyat (USD) Sinyal
                  bitcoin         btc 99546.000000  BEKLE
                 ethereum         eth  2720.660000  BEKLE
                   ripple         xrp     2.660000  BEKLE
                   tether        usdt     1.002000  BEKLE
                   solana         sol   207.400000  BEKLE
              binancecoin         bnb   596.930000  BEKLE
                 usd-coin        usdc     0.999999  BEKLE
                 dogecoin        doge     0.271378  BEKLE
                  cardano         ada     0.773520  BEKLE
             staked-ether       steth  2714.980000  BEKLE
                     tron         trx     0.226772  BEKLE
                chainlink        link    20.590000  BEKLE
          wrapped-bitcoin        wbtc 99566.000000  BEKLE
              avalanche-2        avax    27.270000  BEKLE
            wrapped-steth      wsteth  3229.020000  BEKLE
                      sui 